In [2]:
from pyomexmeta import RDF, eUriType

cellml = """
<model xmlns="http://www.cellml.org/cellml/1.1#" xmlns:cmeta="http://www.cellml.org/metadata/1.0#"
      name="annotation_examples" cmeta:id="annExamples">
  <component name="main">
    <variable cmeta:id="main.Volume" initial_value="100" name="Volume" units="dimensionless" />
    <variable cmeta:id="main.MembraneVoltage" initial_value="-80" name="MembraneVoltage" units="dimensionless" />
    <variable cmeta:id="main.ReactionRate" initial_value="1" name="ReactionRate" units="dimensionless" />
  </component>
</model>
"""

rdf = RDF()
rdf.set_archive_uri("physical_process.omex")
rdf.set_model_uri("model.cellml")
annot_editor = rdf.to_editor(cellml, generate_new_metaids=False, sbml_semantic_extraction=False)

# fma:14515 = hepatocyte
# fma:66836 = portion of cytosol
with annot_editor.new_physical_entity() as cytosol:
    cytosol \
        .about("cytosol", eUriType.LOCAL_URI) \
        .identity("FMA:66836") \
        .is_part_of("FMA:14515")
    
# http://purl.obolibrary.org/obo/PR_000003767 = alcohol dehydrogenase 1A
with annot_editor.new_physical_entity() as mediator:
    mediator \
        .about("mediator", eUriType.LOCAL_URI) \
        .identity("http://purl.obolibrary.org/obo/PR_000003767") \
        .is_part_of("http://omex-library.org/physical_process.omex/model.rdf#cytosol")
    # \
    #keyword arguments to make sure correct overload is chosen
    #    .has_property("protein", eUriType.LOCAL_URI)

# with annot_editor.new_physical_entity() as source:
#     source.about("source-0", eUriType.LOCAL_URI) \
#                    .identity("http://purl.obolibrary.org/obo/PR_000003767") \
#                    .is_part_of("fma:66836") \
#                    .is_part_of("fma:14515")

# CHEBI:16236 = ethanol
with annot_editor.new_physical_entity() as source:
    source \
        .about("source", eUriType.LOCAL_URI) \
        .identity("CHEBI:16236") \
        .is_part_of("http://omex-library.org/physical_process.omex/model.rdf#cytosol")

# CHEBI:15343 = acetaldehyde
with annot_editor.new_physical_entity() as sink:
    sink \
        .about("sink", eUriType.LOCAL_URI) \
        .identity("CHEBI:15343") \
        .is_part_of("http://omex-library.org/physical_process.omex/model.rdf#cytosol")

# opb:OPB_00592 = chemical molar flow rate
# GO:0004022 = alcohol dehydrogenase (NAD+) activity
with annot_editor.new_physical_process() as reaction_rate:
    reaction_rate \
        .about("process-0", eUriType.LOCAL_URI) \
        .is_version_of("GO:0004022") \
        .add_source("source", eUriType.LOCAL_URI, multiplier=1) \
        .add_sink("sink", eUriType.LOCAL_URI, multiplier=1) \
        .add_mediator("mediator", eUriType.LOCAL_URI) \
        .has_property("main.ReactionRate", eUriType.MODEL_URI, "opb:OPB_00592")

print(rdf)
rdf.draw('tmp')
from IPython.display import IFrame
IFrame("tmp.pdf", width=1024, height=720)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix semsim: <http://bime.uw.edu/semsim/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/physical_process.omex/> .
@prefix local: <http://omex-library.org/physical_process.omex/model.rdf#> .

<http://omex-library.org/physical_process.omex/model.cellml#main.ReactionRate>
    bqbiol:isPropertyOf local:process-0 ;
    bqbiol:isVersionOf <https://identifiers.org/opb:OPB_00592> .

local:MediatorParticipant0000
    semsim:hasPhysicalEntityReference local:mediator .

local:Property0000
    bqbiol:isPropertyOf local:cytosol, local:mediator, local:sink, local:source .

local:SinkParticipant0000
    semsim:hasMultiplier "1"^^rdf:double ;
    semsim:hasPhysicalEntityReference local:sink .

local:SourceParticipant0000
    semsim:hasMultiplier "1"^^rdf:double ;
    semsim:hasPhysicalEntityReference local:source .

local:cytosol
    bq